In [1]:
import pandas as pd
import sqlite3, matplotlib
from sqlalchemy import create_engine
import re

Read data from CSV

In [2]:
# read the Park Data
df_pr = pd.read_csv('parks.csv', encoding='utf-8')
# read the Species Data
df_sp = pd.read_csv('species.csv', encoding='utf-8')

C:\Users\guila\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_pr.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [4]:
df_sp.head(5)

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern,NaN
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered,NaN
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN,NaN


In [5]:
df_pr.columns

Index(['Park Code', 'Park Name', 'State', 'Acres', 'Latitude', 'Longitude'], dtype='object')

In [6]:
df_pr.rename(columns={'Park Code':'park_code','Park Name':'park_name','State':'state','Acres':'acres','Latitude':'latitude','Longitude':'longitude'}, inplace=True)

In [7]:
df_pr.columns

Index(['park_code', 'park_name', 'state', 'acres', 'latitude', 'longitude'], dtype='object')

In [8]:
df_sp.columns

Index(['Species ID', 'Park Name', 'Category', 'Order', 'Family',
       'Scientific Name', 'Common Names', 'Record Status', 'Occurrence',
       'Nativeness', 'Abundance', 'Seasonality', 'Conservation Status',
       'Unnamed: 13'],
      dtype='object')

In [9]:
df_sp.drop(['Unnamed: 13'], axis=1, inplace=True)

In [10]:
df_sp.rename(columns={'Species ID':'species_ID','Park Name':'park_name','Category':'category','Order':'order','Family':'family','Scientific Name':'scientific_name','Common Names':'common_names','Record Status':'record_status','Occurence':'occurence','Nativeness':'nativeness','Abundance':'abundance','Seasonability':'seasonability','Conservation Status':'conservation_status'}, inplace=True)

In [11]:
df_sp.columns

Index(['species_ID', 'park_name', 'category', 'order', 'family',
       'scientific_name', 'common_names', 'record_status', 'Occurrence',
       'nativeness', 'abundance', 'Seasonality', 'conservation_status'],
      dtype='object')

In [25]:
df_sp.head()

,species_ID,park_name,category,order,family,scientific_name,common_names,record_status,Occurrence,nativeness,abundance,Seasonality,conservation_status
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN


In [12]:
df_pr= df_pr.iloc[:,0:6]
df_pr.head()

,park_code,park_name,state,acres,latitude,longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


Performing linear interpolation at missing datapoints

In [15]:
missing_data_rows = df_pr[df_pr.isnull().any(1)]
missing_data_rows

,park_code,park_name,state,acres,latitude,longitude


In [16]:
# check if the missing datapoints still exist 
check_missing_data_rows = df_pr[df_pr.isnull().any(1)]
check_missing_data_rows

,park_code,park_name,state,acres,latitude,longitude


In [17]:
df_pr.head()

,park_code,park_name,state,acres,latitude,longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [18]:
df_pr.columns

Index(['park_code', 'park_name', 'state', 'acres', 'latitude', 'longitude'], dtype='object')

In [19]:
df_sp.columns

Index(['species_ID', 'park_name', 'category', 'order', 'family',
       'scientific_name', 'common_names', 'record_status', 'Occurrence',
       'nativeness', 'abundance', 'Seasonality', 'conservation_status'],
      dtype='object')

Save to the sqlite database

In [20]:
engine = create_engine('sqlite:///parks_and_species.db', echo=True)

In [21]:
# save the parks data into sqlite
df_pr.to_sql('parks', con=engine, if_exists='replace')

2019-08-17 16:42:25,431 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-08-17 16:42:25,432 INFO sqlalchemy.engine.base.Engine ()
2019-08-17 16:42:25,434 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-08-17 16:42:25,435 INFO sqlalchemy.engine.base.Engine ()
2019-08-17 16:42:25,437 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("parks")
2019-08-17 16:42:25,438 INFO sqlalchemy.engine.base.Engine ()
2019-08-17 16:42:25,442 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("parks")
2019-08-17 16:42:25,443 INFO sqlalchemy.engine.base.Engine ()
2019-08-17 16:42:25,446 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-08-17 16:42:25,450 INFO sqlalchemy.engine.base.Engine ()
2019-08-17 16:42:25,453 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("parks")
2019-08-17 16:42:25,454 INFO sqlalchemy.engine.base.Engine ()
2019-08-1

In [22]:
# save the divorce data into sqlite
df_sp.to_sql('species', con=engine, if_exists='replace')

2019-08-17 16:42:34,828 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("species")
2019-08-17 16:42:34,829 INFO sqlalchemy.engine.base.Engine ()
2019-08-17 16:42:34,834 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("species")
2019-08-17 16:42:34,836 INFO sqlalchemy.engine.base.Engine ()
2019-08-17 16:42:34,842 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-08-17 16:42:34,843 INFO sqlalchemy.engine.base.Engine ()
2019-08-17 16:42:34,846 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("species")
2019-08-17 16:42:34,847 INFO sqlalchemy.engine.base.Engine ()
2019-08-17 16:42:34,852 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'species' AND type = 'table'
2019-08-17 16:42:34,853 INFO sqlalchemy.engine.base.Engine ()
2019-08-17 16:42:34,855 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("species")
2019-08-17 16:42

In [23]:
engine.execute("SELECT * FROM parks").fetchall()

2019-08-17 16:42:55,713 INFO sqlalchemy.engine.base.Engine SELECT * FROM parks
2019-08-17 16:42:55,714 INFO sqlalchemy.engine.base.Engine ()


[(0, 'ACAD', 'Acadia National Park', 'ME', 47390, 44.35, -68.21),
 (1, 'ARCH', 'Arches National Park', 'UT', 76519, 38.68, -109.57),
 (2, 'BADL', 'Badlands National Park', 'SD', 242756, 43.75, -102.5),
 (3, 'BIBE', 'Big Bend National Park', 'TX', 801163, 29.25, -103.25),
 (4, 'BISC', 'Biscayne National Park', 'FL', 172924, 25.65, -80.08),
 (5, 'BLCA', 'Black Canyon of the Gunnison National Park', 'CO', 32950, 38.57, -107.72),
 (6, 'BRCA', 'Bryce Canyon National Park', 'UT', 35835, 37.57, -112.18),
 (7, 'CANY', 'Canyonlands National Park', 'UT', 337598, 38.2, -109.93),
 (8, 'CARE', 'Capitol Reef National Park', 'UT', 241904, 38.2, -111.17),
 (9, 'CAVE', 'Carlsbad Caverns National Park', 'NM', 46766, 32.17, -104.44),
 (10, 'CHIS', 'Channel Islands National Park', 'CA', 249561, 34.01, -119.42),
 (11, 'CONG', 'Congaree National Park', 'SC', 26546, 33.78, -80.78),
 (12, 'CRLA', 'Crater Lake National Park', 'OR', 183224, 42.94, -122.1),
 (13, 'CUVA', 'Cuyahoga Valley National Park', 'OH', 32

In [24]:
engine.execute("SELECT * FROM species").fetchall()

2019-08-17 16:42:59,609 INFO sqlalchemy.engine.base.Engine SELECT * FROM species
2019-08-17 16:42:59,610 INFO sqlalchemy.engine.base.Engine ()


[(0, 'ACAD-1000', 'Acadia National Park', 'Mammal', 'Artiodactyla', 'Cervidae', 'Alces alces', 'Moose', 'Approved', 'Present', 'Native', 'Rare', 'Resident', None),
 (1, 'ACAD-1001', 'Acadia National Park', 'Mammal', 'Artiodactyla', 'Cervidae', 'Odocoileus virginianus', 'Northern White-Tailed Deer, Virginia Deer, White-Tailed Deer', 'Approved', 'Present', 'Native', 'Abundant', None, None),
 (2, 'ACAD-1002', 'Acadia National Park', 'Mammal', 'Carnivora', 'Canidae', 'Canis latrans', 'Coyote, Eastern Coyote', 'Approved', 'Present', 'Not Native', 'Common', None, 'Species of Concern'),
 (3, 'ACAD-1003', 'Acadia National Park', 'Mammal', 'Carnivora', 'Canidae', 'Canis lupus', 'Eastern Timber Wolf, Gray Wolf, Timber Wolf', 'Approved', 'Not Confirmed', 'Native', None, None, 'Endangered'),
 (4, 'ACAD-1004', 'Acadia National Park', 'Mammal', 'Carnivora', 'Canidae', 'Vulpes vulpes', 'Black Fox, Cross Fox, Eastern Red Fox, Fox, Red Fox, Silver Fox', 'Approved', 'Present', 'Unknown', 'Common', 'Bree